<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Graph_Neural_Networks_(GNNs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch_geometric

In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
import networkx as nx

# Set random seed for reproducibility
torch.manual_seed(42)

# Define a simple graph with 3 nodes and 4 edges
edge_index = torch.tensor([[0, 1, 2, 0], [1, 0, 1, 2]], dtype=torch.long)  # Edge list
# Define node features with two-dimensional feature space
x = torch.tensor([[-1, 1], [0, 0], [1, -1]], dtype=torch.float)

# Create a graph data object
data = Data(x=x, edge_index=edge_index)

# Visualize the graph structure
def visualize_graph(data):
    G = to_networkx(data, to_undirected=True)  # Convert to NetworkX graph
    nx.draw(G, with_labels=True, node_color='lightblue', node_size=500, font_size=10)
    plt.title("Graph Visualization")
    plt.show()

visualize_graph(data)

# Define the Graph Convolutional Network (GCN)
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(2, 4)  # Input features: 2, Output features: 4
        self.conv2 = GCNConv(4, 1)  # Input features: 4, Output features: 1

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)  # First GCN layer
        x = torch.relu(x)  # Apply ReLU activation
        x = self.conv2(x, edge_index)  # Second GCN layer
        return x

# Initialize the model
model = GCN()

# Forward pass through the model
out = model(data)

# Print node embeddings after GCN layers
print("Node embeddings after GCN layers:")
print(out)